In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
from prefect.blocks.system import Secret
from catnip.fla_redshift import FLA_Redshift
from typing import Dict
from concurrent.futures import ThreadPoolExecutor

In [37]:
def get_redshift_credentials() -> Dict:

    cred_dict = {
        "dbname": Secret.load("stellar-redshift-db-name").get(),
        "host": Secret.load("stellar-redshift-host").get(),
        "port": 5439,
        "user": Secret.load("stellar-redshift-user-name").get(),
        "password": Secret.load("stellar-redshift-password").get(),

        "aws_access_key_id": Secret.load("fla-s3-aws-access-key-id-east-1").get(),
        "aws_secret_access_key": Secret.load("fla-s3-aws-secret-access-key-east-1").get(),
        "bucket": Secret.load("fla-s3-bucket-name-east-1").get(),
        "subdirectory": "us-east-1",

        "verbose": False,
    }

    return cred_dict

with ThreadPoolExecutor(1) as pool:
    rs_creds = pool.submit(lambda: get_redshift_credentials()).result()

In [45]:
q = """
select *
from custom.cte_event_descriptions"""

df = FLA_Redshift(**rs_creds).query_warehouse(sql_string=q)
df['event_name_year'] = df.apply(lambda row: row['event_name'] + str(row['event_datetime'].year), axis = 1)

num_df = df.groupby("event_name_year").nunique()['event_datetime']
num_df = num_df.to_frame()
num_df.reset_index(inplace = True)
num_df.rename(columns = {"event_datetime":"num_events"}, inplace = True)
max_df = df.groupby("event_name_year").max()['event_datetime']
max_df = max_df.to_frame()
max_df.reset_index(inplace = True)
max_df.rename(columns = {"event_datetime":"max_date"}, inplace = True)
min_df = df.groupby("event_name_year").min()['event_datetime']
min_df = min_df.to_frame()
min_df.reset_index(inplace = True)
min_df.rename(columns = {"event_datetime":"min_date"}, inplace = True)

final_df = num_df.merge(max_df, how = 'left', on = 'event_name_year')
final_df = final_df.merge(min_df, how = 'left', on = 'event_name_year')
final_df['event_range'] = final_df.apply(lambda row: row['min_date'].strftime("%m/%d/%y") if row['min_date'] == row['max_date'] else str(row['min_date'].strftime("%m/%d/%y"))+ ' - ' + str(row['max_date'].strftime("%m/%d/%y")), axis = 1)
final_df = final_df[['event_name_year', 'num_events', 'event_range']]

In [46]:
df.merge(final_df, how = 'left', on = 'event_name_year')

,seatgeek_product_id,event_type,product_description,event_datetime,event_name,primary_event_type,secondary_event_type,event_name_parameter,start_time,start_time_tableau,processed_date,event_name_year,num_events,event_range
0,bf16b703-ca41-ee11-8445-c1174e74cf37,3PE Premium,23E1### - Travis Scott PREMIUM,2000-11-26 19:00:00,Travis Scott,Concerts,Hiphop/Rap,2000-11-26 - Travis Scott - 7 PM,7:00 PM,7 PM,2023-11-09 15:51:08.461788,Travis Scott2000,3,11/26/00 - 12/02/00
1,be89ca74-63bf-4800-8e09-756547b60ab0,3PE Premium,23E1### - Travis Scott PREMIUM,2000-12-02 19:00:00,Travis Scott,Concerts,Hiphop/Rap,2000-12-02 - Travis Scott - 7 PM,7:00 PM,7 PM,2023-11-09 15:51:08.461788,Travis Scott2000,3,11/26/00 - 12/02/00
2,777bc0b4-6cf9-ed11-8445-c38f5fa7c736,3PE Upsells,ENC08##PP - Monster Jam Pit Pass - August 2023,2023-08-12 13:00:00,Monster Jam,Sports,Motorsports,2023-08-12 - Monster Jam - 1 PM,1:00 PM,1 PM,2023-11-09 15:51:08.461788,Monster Jam2023,3,08/12/23 - 08/13/23
3,a176d89c-69f9-ed11-8445-c38f5fa7c736,3PE Premium,23E08## - Monster Jam PREMIUM - August 2023,2023-08-12 19:00:00,Monster Jam,Sports,Motorsports,2023-08-12 - Monster Jam - 7 PM,7:00 PM,7 PM,2023-11-09 15:51:08.461788,Monster Jam2023,3,08/12/23 - 08/13/23
4,3bddc7cc-6cf9-ed11-8445-c38f5fa7c736,3PE FELD Admission,ENC08## - Monster Jam - August 2023,2023-08-13 13:00:00,Monster Jam,Sports,Motorsports,2023-08-13 - Monster Jam - 1 PM,1:00 PM,1 PM,2023-11-09 15:51:08.461788,Monster Jam2023,3,08/12/23 - 08/13/23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,bca17f62-3a53-ee11-8445-9567206ab876,3PE CONCERT Admission,ENC0525 - NF Hope Tour,2024-05-25 20:00:00,NF,Concerts,Hiphop/Rap,2024-05-25 - NF - 8 PM,8:00 PM,8 PM,2023-11-09 15:51:08.461788,NF2024,1,05/25/24
95,3f6f9c65-a22c-ee11-8445-dcbdddd421c0,3PE Premium,24E0606 - Luis Miguel PREMIUM,2024-06-06 20:00:00,Luis Miguel,Concerts,Latin,2024-06-06 - Luis Miguel - 8 PM,8:00 PM,8 PM,2023-11-09 15:51:08.461788,Luis Miguel2024,1,06/06/24
96,09df950a-d9d7-ed11-842b-e72a3871f1c7,3PE Premium,23E0804 - Pepe Aguilar PREMIUM,2024-06-29 20:00:00,Pepe Aguilar,Concerts,Latin,2024-06-29 - Pepe Aguilar - 8 PM,8:00 PM,8 PM,2023-11-09 15:51:08.461788,Pepe Aguilar2024,1,06/29/24
97,9305feac-c466-ee11-8445-cf3cdedc965c,3PE Premium,24E0722 - Zach Bryan Premium,2024-07-22 19:00:00,Zach Bryan,Concerts,Country,2024-07-22 - Zach Bryan - 7 PM,7:00 PM,7 PM,2023-11-09 15:51:08.461788,Zach Bryan2024,1,07/22/24
